- Here we explore the use of `vtki`to create 3D objects we then display using `ipyvolume`.
- In the future we might want to use `itkwidgets` to display 3d objects. See https://github.com/InsightSoftwareConsortium/itk-jupyter-widgets/issues/112


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import sys
sys.path.append("../../../")

import anamic

import numpy as np
import matplotlib.pyplot as plt
import ipyvolume as ipv
import vtki

In [5]:
n_pf = 11
mt_length_nm = 200  # nm
taper_length_nm = 0  # nm

# Create a MicrotubuleSimulator object.
dimers = anamic.simulator.dimers_builder(n_pf, mt_length_nm, taper_length_nm)

# Set parameters for the image generation.
parameters = {}
parameters['labeling_ratio'] = 0.1  # from 0 to 1

parameters['pixel_size'] = 110  # nm/pixel
parameters['x_offset'] = 1500  # nm
parameters['y_offset'] = 1500  # nm

parameters['psf_size'] = 135  # nm

parameters['signal_mean'] = 700
parameters['signal_std'] = 100
parameters['bg_mean'] = 500
parameters['bg_std'] = 24
parameters['noise_factor'] = 1

parameters['snr_line_width'] = 3  # pixel

ms = anamic.simulator.MicrotubuleSimulator(dimers)
ms.parameters.update(parameters)

# Build the geometry.
ms.build_positions(apply_random_z_rotation=True, show_progress=True)
ms.label()
ms.project()
ms.random_rotation_projected()

100%|██████████| 24/24 [00:00<00:00, 4461.63it/s]


In [7]:
import matplotlib

def make_sphere(radius, center, direction=[0, 0, 1], sphere_resolution=50):
    sphere = vtki.Sphere(radius, center, direction,
                         theta_resolution=sphere_resolution, phi_resolution=sphere_resolution)

    x = sphere.points[:, 0]
    y = sphere.points[:, 1]
    z = sphere.points[:, 2]

    # The faces returned by `PolyData` can be different
    # than triangle, so we assume `vtki.Sphere.faces` returns
    # only triangles.
    triangles = np.array([sphere.faces[1::4], sphere.faces[2::4], sphere.faces[3::4]]).T
    
    return x, y, z, triangles

radius = 4
direction = [0, 0, 1]
sphere_resolution = 30
color_feature_name = 'pf'
cmap_name="tab20c"

# Only show visible dimers
selected_dimers = ms.positions[ms.positions['visible'] == True]

# Build the dimer's 3D objects.
objects = []
positions = selected_dimers
for _, dimer in positions.iterrows():
    center = dimer[['x', 'y', 'z']].values.astype('float32')
    obj = make_sphere(radius, center, direction, sphere_resolution)
    objects.append(obj)
    
# Display the 3D scene

if color_feature_name:
    # TODO: that code should be much simpler...
    cmap = matplotlib.cm.get_cmap(cmap_name)
    categories = selected_dimers[color_feature_name].unique()
    color_indices = cmap([i / len(categories) for i in categories])

    colors = np.zeros((len(selected_dimers[color_feature_name]), 4))
    for color_index in range(len(categories)):
        colors[selected_dimers[color_feature_name] == categories[color_index]] = color_indices[color_index]
else:
    colors = np.repeat('#e4191b', selected_dimers.shape[0])

fig = ipv.figure()

for (x, y, z, triangles), color in zip(objects, colors):
    ipv.plot_trisurf(x, y, z, triangles=triangles, color=color)

ipv.squarelim()
ipv.show()

---

In [6]:
# Display the PDB model of the alpha/beta-tubulin dimers.

model_path = Path('/home/hadim/Documents/Code/Postdoc/Data/tubulin-dimer.stl')
dimer = vtki.PolyData(str(model_path))

# Move dimer to center of mass
mass_center = dimer.points.mean(axis=0)
dimer.translate(mass_center * -1)

x = dimer.points[:, 0]
y = dimer.points[:, 1]
z = dimer.points[:, 2]

# The faces returned by `PolyData` can be different
# than triangle, so we assume `vtki.Sphere.faces` returns
# only triangles.
triangles = np.array([dimer.faces[1::4], dimer.faces[2::4], dimer.faces[3::4]]).T

ipv.figure()
ipv.plot_trisurf(x, y, z, triangles=triangles, color='orange')
ipv.xyzlim(-50, 50)
ipv.show()